# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# part 1 files
city_weather_path = "../output_data/updated_cities_weather_data.csv"

# Read the city weather data

city_weather = pd.read_csv(city_weather_path)

city_weather.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Christchurch,-43.5333,172.6333,4.12,89,75,6.69,NZ,1663071964
1,1,Avarua,-21.2078,-159.7750,23.03,83,40,3.09,CK,1663071965
2,2,Mandalgovi,45.7625,106.2708,22.48,23,100,3.33,MN,1663071967
3,3,Atuona,-9.8000,-139.0333,24.50,67,1,6.40,PF,1663071968
4,4,Haines Junction,60.7522,-137.5108,-1.03,93,99,1.54,CA,1663071969


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
locations = city_weather[['Lat', 'Lng']]
weights = city_weather['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights,max_intensity=100, point_radius=10))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 27 degrees but higher than 21

# Wind speed less than 4.5 m/s

# Zero cloudiness

condition_maxtemp =  (city_weather["Max Temp"]<27)
condition_mintemp =  (city_weather["Max Temp"]>21)
condition_wind_speed =  (city_weather["Wind Speed"]<4.5)
condition_cloudiness =  (city_weather["Cloudiness"]==0)

hotel_df = city_weather.loc[condition_maxtemp & condition_mintemp & 
                                          condition_wind_speed & condition_cloudiness].dropna()
hotel_df 

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,Alice Springs,-23.7000,133.8833,21.75,37,0,3.09,AU,1663071985
223,223,Wilmington Island,32.0036,-80.9737,24.43,72,0,1.54,US,1663072270
287,287,Guerrero Negro,27.9769,-114.0611,21.52,89,0,3.86,MX,1663072361
336,336,Porterville,36.0652,-119.0168,24.86,59,0,3.09,US,1663072434
347,347,Aksu,41.1231,80.2644,25.36,24,0,4.40,CN,1663072451
370,370,Fort Walton Beach,30.4058,-86.6188,22.77,87,0,3.60,US,1663072482
380,380,Makakilo City,21.3469,-158.0858,25.18,75,0,4.12,US,1663072496
451,451,Estelle,29.8458,-90.1067,24.44,74,0,2.06,US,1663072569
578,578,De Aar,-30.6497,24.0123,21.02,12,0,0.57,ZA,1663072767


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
 
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    lat = row['Lat']
    
    lng = row['Lng']

    # add keyword to params dict
    params['location'] =  f"{lat}, {lng}"

    # assemble url and make API request
    
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
      
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
16,16,Alice Springs,-23.7000,133.8833,21.75,37,0,3.09,AU,1663071985,Desert Palms Alice Springs
223,223,Wilmington Island,32.0036,-80.9737,24.43,72,0,1.54,US,1663072270,Breezeway Studio Manor LLC
287,287,Guerrero Negro,27.9769,-114.0611,21.52,89,0,3.86,MX,1663072361,"Casa Laguna, Bed & Breakfast"
336,336,Porterville,36.0652,-119.0168,24.86,59,0,3.09,US,1663072434,"Holiday Inn Express & Suites Porterville, an I..."
347,347,Aksu,41.1231,80.2644,25.36,24,0,4.40,CN,1663072451,Pudong Holiday Hotel
370,370,Fort Walton Beach,30.4058,-86.6188,22.77,87,0,3.60,US,1663072482,Best Western Ft. Walton Beachfront
380,380,Makakilo City,21.3469,-158.0858,25.18,75,0,4.12,US,1663072496,Marriott's Ko Olina Beach Club
451,451,Estelle,29.8458,-90.1067,24.44,74,0,2.06,US,1663072569,Ohio
578,578,De Aar,-30.6497,24.0123,21.02,12,0,0.57,ZA,1663072767,Nyibiba Guesthouse


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map


marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))